In [1]:
# Script to calculate originality and  application data
# This script prepares the classification data before merging
# Jan 16th, 2020
# Multiple classifications generate an error into the Generality calculation
# I am creating a version with the first class only 

# Jan 7th, 2020


In [2]:
import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler

import pandas as pd
import numpy as np
import re



In [3]:
# from the data dictionary

# patent_id: patent number
# field_id:  WIPO technology field ID as derived from crosswalk 
#            http://www.wipo.int/export/sites/www/ipstats/en/statistics/patents/xls/ipc_technology.xls
# sequence:  order in which WIPO technology field appears on patent


In [4]:
wipo = 'data/wipo.tsv'
dst= 'data/wipo.csv'

In [5]:
# 
df=pd.read_csv(wipo, sep='\t', dtype='object')

# sample_size=1000
# wipo_df=sampler(wipo, sample_size, sep='\t')


In [6]:
# # # Keep this for reference!
# # # As of Dec 31st, 2019, I compared the clean to the raw version of citation and patent ids

# # #stripping non-desired characters but keeping the originals for later check - only three changes in citation_id

# wipo_df['patent_id_raw']=wipo_df['patent_id']

# cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
# wipo_df['patent_id']=wipo_df['patent_id'].apply(cleaning_patent)

In [7]:
#first, construct a table with wipo categories in columns, patent level
#second, 

In [8]:
df.dtypes

patent_id    object
field_id     object
sequence     object
dtype: object

In [9]:
df.groupby('sequence').patent_id.count()

sequence
0     6441314
1     2032186
10         95
11         26
12         26
13         12
2      449147
3       96764
4       22529
5        6794
6        2401
7        1053
8         554
9         178
Name: patent_id, dtype: int64

In [10]:
df=df[df['sequence']=='0'][['patent_id', 'field_id']]

In [11]:
df.field_id.value_counts()

6     610168
1     406495
4     367556
8     335881
13    331891
10    304120
2     301882
9     296296
32    272396
14    229166
3     206962
29    191855
27    177452
35    176277
31    175502
33    160647
25    155111
26    149727
23    145420
15    139241
28    132710
19    125275
17    121192
5     119108
34    116886
12    110166
16    103783
20    100333
21     90063
7      75837
30     63740
24     62631
18     41730
11     30087
22     13728
Name: field_id, dtype: int64

In [12]:
df=df.add_prefix('wipo_')

In [13]:
df['wipo_field_id']=df['wipo_field_id'].apply('{:0>2}'.format)

In [14]:
df.rename(columns={ df.columns[0]: "id" }, inplace = True)

In [15]:
df.head()

,id,wipo_field_id
4,10000122,01
6,10000133,01
7,10000134,01
9,10000138,01
15,10000167,01


In [16]:
class_system='wipo_field_id'

# df_classes=pd.read_csv('data/classes.csv', usecols=['id','sector_title'])
df_classes=pd.read_csv('data/classes.csv')

df_classes.rename(columns={'id': 'class_id'}, inplace=True)
df_classes=df_classes[df_classes['system']==class_system]

df_classes['class_id']=df_classes['class_id'].apply('{:0>2}'.format)
# df_classes=df_classes.reset_index()

df=df.merge(df_classes, left_on='wipo_field_id', right_on='class_id')


# rank=df.reset_index().groupby(class_system).count()['id'].reset_index()
# rank.rename(columns={class_system: 'id', 'id':class_system}, inplace=True)

# df_classes=df_classes[df_classes['system']==class_system]

# rank=rank.merge(df_classes, left_on='id', right_on='id')
# display.display(rank.groupby('sector_title').sum().sort_values(by=class_system, ascending=False))

In [17]:
sector_id=df_classes.groupby('sector_title').count().reset_index().iloc[:,:1].reset_index()

In [18]:
df.head()

,id,wipo_field_id,system,class_id,sector_title,field_title
0,10000122,01,wipo_field_id,01,Electrical Eng,"Electrical machinery, apparatus, energy"
1,10000133,01,wipo_field_id,01,Electrical Eng,"Electrical machinery, apparatus, energy"
2,10000134,01,wipo_field_id,01,Electrical Eng,"Electrical machinery, apparatus, energy"
3,10000138,01,wipo_field_id,01,Electrical Eng,"Electrical machinery, apparatus, energy"
4,10000167,01,wipo_field_id,01,Electrical Eng,"Electrical machinery, apparatus, energy"


In [19]:
sector_id

,index,sector_title
0,0,Chemistry
1,1,Electrical Eng
2,2,Instruments
3,3,Mechanical Eng
4,4,Other fields
5,5,Plant


In [20]:
df=df.merge(sector_id, left_on='sector_title', right_on='sector_title')

In [21]:
df=df[['id','index', 'wipo_field_id']]

In [22]:
df.rename(columns={'index': 'wipo_sector_id'}, inplace=True)

In [ ]:
df_classes['wipo_field_id']=df_classes['wipo_field_id'].apply('{:0>2}'.format)


In [23]:
df.head()

,id,wipo_sector_id,wipo_field_id
0,10000122,1,01
1,10000133,1,01
2,10000134,1,01
3,10000138,1,01
4,10000167,1,01


In [24]:
df.set_index('id').to_csv(dst)